In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import linear_model

In [2]:
# Reading csv file
df_train =  pd.read_csv("train_fold.csv")
df_test = pd.read_csv(r'E:\CS\Practice\kaggle\comp\tps-aug-22\data\test.csv')
sample_submission = pd.read_csv(r'E:\CS\Practice\kaggle\comp\tps-aug-22\data\sample_submission.csv')
df_train.head(3)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,kfold
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0,4
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0,0


In [3]:
corr_matrix = df_train.corr()

In [4]:
corr_matrix["failure"].sort_values(ascending=False)

failure           1.000000
loading           0.129089
measurement_17    0.033905
measurement_5     0.018079
measurement_8     0.017119
measurement_7     0.016787
measurement_2     0.015808
measurement_6     0.014791
measurement_0     0.009646
attribute_2       0.006337
measurement_14    0.006211
measurement_12    0.004398
measurement_3     0.003577
measurement_16    0.002237
kfold             0.000130
measurement_10   -0.001515
measurement_13   -0.001831
measurement_15   -0.003544
measurement_9    -0.003587
measurement_11   -0.004801
id               -0.007545
measurement_4    -0.010488
measurement_1    -0.010810
attribute_3      -0.019222
Name: failure, dtype: float64

In [5]:
# useful_features = corr_matrix.index[corr_matrix["failure"] > 0]
# df_train[useful_features]

In [6]:
# useful_features = []
# for f in corr_matrix["failure"]:
#     if f > 0:
#         useful_features.append(corr_matrix.index)
# print(useful_features)

In [7]:
# # Seperate target from predictors
# y = df_train.failure
# df_train.drop(['failure'], axis=1, inplace=True)
# df_train

In [8]:
# all columns are features except id, failureand kfold columns
features = [
    f for f in df_train.columns if f not in ('id','kfold','failure')
]

In [9]:
# Categorical columns in the training data
object_cols = [col for col in df_train.columns if df_train[col].dtype=='object']
print('Categoricaal columns are:\n',object_cols)

Categoricaal columns are:
 ['product_code', 'attribute_0', 'attribute_1']


In [10]:
df_test = df_test[features]

In [11]:
for fold in range(5):
    X_train = df_train[df_train.kfold != fold].reset_index(drop=True)
    X_valid = df_train[df_train.kfold == fold].reset_index(drop=True)
    X_test = df_test.copy()
    
    y_train = X_train.failure
    y_valid = X_valid.failure
    
    X_train.drop(['failure'], axis = 1, inplace=True)
    X_valid.drop(['failure'], axis = 1, inplace=True)
    
    # Select numerical columns
    numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
    
    # Select categorical columns
    categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
    
    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='mean')
    
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy ='most_frequent')),
        ('onehot',OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Bundle preprocessing for numerical and categorical daya
    preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
    
    # Define model
    model = linear_model.LogisticRegression()
    
    # Bundle preprocessing and modeling code in a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
    
    # Preprocessing of training data, fit model 
    clf.fit(X_train, y_train)
    
    # Preprocessing of validation data, get predictions
    preds = clf.predict(X_valid)
    
    # roc_auc_metric
    score = roc_auc_score(y_valid,preds)
    
    # print score
    print(fold, score)

0 0.501209135075817
1 0.5008494221560432
2 0.5010884278921809
3 0.5003229750080374
4 0.5004909135518367


In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
for fold in range(5):
    X_train = df_train[df_train.kfold != fold].reset_index(drop=True)
    X_valid = df_train[df_train.kfold == fold].reset_index(drop=True)
    X_test = df_test.copy()
    
    y_train = X_train.failure
    y_valid = X_valid.failure
    
    X_train.drop(['failure'], axis = 1, inplace=True)
    X_valid.drop(['failure'], axis = 1, inplace=True)
    
    # Select numerical columns
    numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
    
    # Select categorical columns
    categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
    
    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='mean')
    
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy ='most_frequent')),
        ('onehot',OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Bundle preprocessing for numerical and categorical daya
    preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
    
    # Define model
    model = DecisionTreeClassifier(random_state=0)
    
    # Bundle preprocessing and modeling code in a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
    
    # Preprocessing of training data, fit model 
    clf.fit(X_train, y_train)
    
    # Preprocessing of validation data, get predictions
    preds = clf.predict(X_valid)
    
    # roc_auc_metric
    score = roc_auc_score(y_valid,preds)
    
    # print score
    print(fold, score)

0 0.4924872985789012
1 0.5015165231222186
2 0.5163541261273457
3 0.5111831418467317
4 0.502120382747593


In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
for fold in range(5):
    X_train = df_train[df_train.kfold != fold].reset_index(drop=True)
    X_valid = df_train[df_train.kfold == fold].reset_index(drop=True)
    X_test = df_test.copy()
    
    y_train = X_train.failure
    y_valid = X_valid.failure
    
    X_train.drop(['failure'], axis = 1, inplace=True)
    X_valid.drop(['failure'], axis = 1, inplace=True)
    
    # Select numerical columns
    numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
    
    # Select categorical columns
    categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
    
    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='mean')
    
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy ='most_frequent')),
        ('onehot',OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Bundle preprocessing for numerical and categorical daya
    preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
    
    # Define model
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    
    # Bundle preprocessing and modeling code in a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
    
    # Preprocessing of training data, fit model 
    clf.fit(X_train, y_train)
    
    # Preprocessing of validation data, get predictions
    preds = clf.predict(X_valid)
    
    # roc_auc_metric
    score = roc_auc_score(y_valid,preds)
    
    # print score
    print(fold, score)

0 0.5000844468572118
1 0.5002034721399685
2 0.49940248565965584
3 0.5003229750080374
4 0.49976099426386233


In [102]:
final_predictions = []
for fold in range(5):
    X_train = df_train[df_train.kfold != fold].reset_index(drop=True)
    X_valid = df_train[df_train.kfold == fold].reset_index(drop=True)
    X_test = df_test.copy()
    
    y_train = X_train.failure
    y_valid = X_valid.failure
    
#     X_train.drop(['failure'], axis = 1, inplace=True)
#     X_valid.drop(['failure'], axis = 1, inplace=True)
    
    X_train.drop(['failure','id','kfold'], axis = 1, inplace=True)
    X_valid.drop(['failure','id','kfold'], axis = 1, inplace=True)
    
    # Select numerical columns
    numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
    
    # Select categorical columns
    categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
    
    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='most_frequent')
    
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy ='most_frequent')),
        ('onehot',OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Bundle preprocessing for numerical and categorical daya
    preprocessor = ColumnTransformer(transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
    
    # Define model
    model = DecisionTreeClassifier(random_state=fold)
    
    # Bundle preprocessing and modeling code in a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
    
    # Preprocessing of training data, fit model 
    clf.fit(X_train, y_train)
    
    # Preprocessing of validation data, get predictions
    preds = clf.predict(X_valid)
    test_preds = clf.predict_proba(X_test)[:,1]
    
    # roc_auc_metric
    score = roc_auc_score(y_valid,preds)
    
    # Appending test
    final_predictions.append(test_preds)
    
    # print score
    print(fold, score)

0 0.5059898007667943
1 0.5049755495016837
2 0.5038710468874262
3 0.5216412291240122
4 0.5162959610145688


In [103]:
df_test.shape

(20775, 25)

In [104]:
final_predictions

[array([0., 1., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 1., 0., ..., 0., 0., 0.]),
 array([0., 0., 1., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.])]

In [95]:
# prediction = np.column_stack(final_predictions)

In [106]:
prediction = np.column_stack(final_predictions)
prediction.shape

(20775, 5)

In [92]:
pred = np.column_stack(final_predictions)
pred.shape

(20775, 5)

In [86]:
# pred_list = []
# for l in final_predictions:
#     pred_list += 1

# print(pred_list)

In [87]:
sample_submission.failure = pred
sample_submission

ValueError: Length of values (5) does not match length of index (20775)